In [ ]:
import string
import re
from collections import Counter
import editdistance
import numpy as np
import tkinter as tk
from tkinter import simpledialog

class Autocorrection(object):
    def __init__(self, filename):
        with open(filename, "r", encoding="utf-8") as file:
            word = []
            lines = file.readlines()
            for line in lines:
                word += re.findall(r'\w+', line.lower())

        self.vocabulary = set(word)
        self.counts_of_word = Counter(word)
        self.total_words = float(sum(self.counts_of_word.values()))
        self.prob_of_word = {w: self.counts_of_word[w] / self.total_words for w in self.counts_of_word.keys()}

    def generate_edit1_candidates(self, word):
        letter = string.ascii_lowercase
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        insert = [l + c + r for l, r in splits for c in letter]
        delete = [l + r[1:] for l, r in splits if r]
        replace = [l + c + r[1:] for l, r in splits if r for c in letter]
        swap = [l + r[1] + r[0] + r[2:] for l, r in splits if len(r) > 1]

        return set(replace + insert + delete + swap)

    def generate_edit2_candidates(self, word):
        return [e2 for e1 in self.generate_edit1_candidates(word) for e2 in self.generate_edit1_candidates(e1)]

    def calculate_common_prefix_length(self, word1, word2):
        common_len = 0
        for i, (c1, c2) in enumerate(zip(word1, word2)):
            if c1 == c2:
                common_len += 1
            else:
                break
        return common_len

    def calculate_custom_score(self, suggestion, original_word):
        replace_weight = 1
        insert_weight = 2
        delete_weight = 3
        swap_weight = 4

        distance = editdistance.eval(suggestion, original_word)

        if len(suggestion) == len(original_word):  # Replace
            common_prefix_len = self.calculate_common_prefix_length(suggestion, original_word)
            score = distance * replace_weight + (len(original_word) - common_prefix_len)
        elif len(suggestion) == len(original_word) + 1:  # Insert
            common_prefix_len = self.calculate_common_prefix_length(suggestion, original_word)
            score = distance * insert_weight + (len(original_word) - common_prefix_len)
        elif len(suggestion) == len(original_word) - 1:  # Delete
            score = distance * delete_weight
        else:  # Swap
            score = distance * swap_weight

        return score

    def get_corrected_spelling(self, word):
        if word in self.vocabulary:
            print(f"{word} is already correctly spelt")
            return

        suggestions = self.generate_edit1_candidates(word) or self.generate_edit2_candidates(word) or [word]
        best_guesses = [w for w in suggestions if w in self.vocabulary]

        best_guesses.sort(key=lambda w: self.calculate_custom_score(w, word))

        return [(w, self.prob_of_word[w]) for w in best_guesses]



In [ ]:

class AutoCorrectApp(tk.Tk):
    def __init__(self):
        super().__init__()
        abc='corpus.txt'

        self.checker = Autocorrection(abc)

        self.title("AutoCorrect Application")

        # Header
        header = tk.Label(self, text="AutoCorrect Application", font=("Helvetica", 16, "bold"))
        header.pack(pady=10)

        # Instructions
        instructions = tk.Label(self, text="Type in the box below. Suggestions will appear as you type.")
        instructions.pack(pady=5)

        # Input box
        input_frame = tk.Frame(self)
        input_frame.pack(pady=10)
        input_label = tk.Label(input_frame, text="Input Text:")
        input_label.pack(side="left")
        self.input_box = tk.Text(input_frame, wrap="word", height=10, width=50)
        self.input_box.pack(side="left", padx=5)

        # Suggestions box
        suggestions_frame = tk.Frame(self)
        suggestions_frame.pack(pady=10)
        suggestions_label = tk.Label(suggestions_frame, text="Suggestions:")
        suggestions_label.pack(side="left")
        self.suggestion_listbox = tk.Listbox(suggestions_frame, height=10, width=50)
        self.suggestion_listbox.pack(side="left", padx=5)

        self.input_box.bind("<KeyRelease>", self.handle_key_release)
        self.input_box.bind("<space>", self.handle_space_bar_press)
        self.suggestion_listbox.bind("<<ListboxSelect>>", self.handle_listbox_selection)

        self.current_word = ""

    def handle_key_release(self, event):
        cursor_position = self.input_box.index(tk.INSERT)
        current_line = self.input_box.get("insert linestart", cursor_position).split()
        self.current_word = current_line[-1] if current_line else ""
        self.after(2000, self.show_autocorrect_suggestions)

    def handle_space_bar_press(self, event):
        if self.suggestion_listbox.curselection():
            return

        word = self.current_word.lower()
        corrections = self.checker.get_corrected_spelling(word)

        if corrections:
            probs = np.array([c[1] for c in corrections])
            best_ix = np.argmax(probs)
            correct = corrections[best_ix][0]
            highest_prob_word = correct

            if highest_prob_word != word:
                self.input_box.delete(f"insert-{len(word)}c", "insert")
                self.input_box.insert("insert", highest_prob_word + " ")

            print(f"Did you mean {highest_prob_word}?")

    def show_autocorrect_suggestions(self):
        word = self.current_word.lower()
        corrections = self.checker.get_corrected_spelling(word)
        self.suggestion_listbox.delete(0, tk.END)

        if corrections:
            for correction in corrections:
                self.suggestion_listbox.insert(tk.END, correction[0])
    
    def handle_listbox_selection(self, event):
        if not self.suggestion_listbox.curselection():
            return
        selected_word = self.suggestion_listbox.get(self.suggestion_listbox.curselection())
        self.input_box.delete(f"insert-{len(self.current_word)}c", "insert")
        self.input_box.insert("insert", selected_word + " ")
        self.suggestion_listbox.delete(0, tk.END)

    def run(self):
        self.mainloop()


In [ ]:

if __name__ == "__main__":
    app = AutoCorrectApp()
    app.run()
